In [1]:
import sys
from ezr import *

In [2]:
import random
import matplotlib.pyplot as plt
import numpy as np
import os

config_folder = 'data/optimize/config'
datasets = [os.path.join(config_folder, file) for file in os.listdir(config_folder) if file.endswith('.csv')]

In [19]:
import random

def activeLearning(self: DATA, score=lambda B, R: B - R, generate=None, faster=True):
    def ranked(rows):
        return self.clone(rows).distances().rows

    def todos(todo):
        if faster:  # Apply sorting heuristics to a small buffer at the start of "todo"
            n = the.buffer // 2
            return todo[:n] + todo[2 * n: 3 * n], todo[3 * n:] + todo[n:2 * n]
        else:  # Apply sorting heuristics to all of "todo"
            return todo, []

    def guess(todo: rows, done: rows) -> rows:
        cut = int(.5 + len(done) ** the.cut)
        best = self.clone(done[:cut])
        rest = self.clone(done[cut:])
        a, b = todos(todo)
        if generate:
            return self.neighbors(generate(best, rest), a) + b
        else:
            key = lambda r: score(best.loglike(r, len(done), 2), rest.loglike(r, len(done), 2))
            return sorted(a, key=key, reverse=True) + b

    def guess_all(todo: rows, done: rows) -> rows:
        cut = int(.5 + len(done) ** the.cut)
        best = self.clone(done[:cut])
        rest = self.clone(done[cut:])
        key = lambda r: best.loglike(r, len(done), 2)
        return sorted(todo, key=key, reverse=True)

    def loop(todo: rows, done: rows) -> rows:
        iteration_results = []
        while len(todo) > 2 and len(done) < the.Last:
            ranking_result = model_ranking(todo, done)
            print(f'{the.iter}. {ranking_result}')
            iteration_results.append(ranking_result)
            top, *todo = guess(todo, done)
            done += [top]
            done = ranked(done)
            the.iter += 1
        iteration_results.append(model_ranking(todo, done))
        return done, iteration_results

    todo, done = self.rows[the.label:], ranked(self.rows[:the.label])

    if the.branch == True:
        todo, done = self.branch(used=[])
        if the.Last == 0: return done

    def model_ranking(todo, done):
        top_real = ranked(self.rows)[:100]
        top_guess = guess_all(self.rows, done)[:200]
        common_elements = [item for item in top_real if item in top_guess]
        return len(common_elements)

    the.iter = 0
    return loop(todo, done)

In [7]:
# Load dataset and initialize
dataset = 'data/optimize/config/SS-N.csv'
d = DATA().adds(csv(dataset))

def exploit(B, R):
    return B

def explore(B, R):
    return (B + R) / (abs(B - R) + 10 ** -30)

the.branch = True
result = d.shuffle().activeLearning(score = lambda B, R: exploit(B, R))

In [6]:
print(done)

[[0, 0, 1, 0, 0.7, 17, 0.8, 34, 3, 167, 34, 86, 334, 36.6, 67, 836, 0.7, 40.666, 494.9090909], [1, 0, 0, 0, 1.4, 17, 0.4, 17, 6, 84, 34, 43, 334, 23.3, 34, 418, 1.4, 47.339, 1855.324074], [0, 1, 0, 0, 1.4, 17, 0.8, 17, 3, 84, -33, 86, 667, 36.6, 67, 836, 1.4, 41.202, 1440.873563], [0, 0, 1, 1, 1.3, 12, 0.8, 39, 2, 173, -84, 83, 658, 14.3, 39, 0, 0.3, 52.391, 1039.389008]]


In [13]:
todo, done = d.rows[the.label:], d.rows[:the.label]

# Exploit with Random Initialization

In [12]:
# Load dataset and initialize
dataset = 'data/optimize/misc/auto93.csv'
d = DATA().adds(csv(dataset))

print(dataset)
print(len(d.rows))

def exploit(B, R):
    return B

def explore(B, R):
    return (B + R) / (abs(B - R) + 10 ** -30)

the.Last = 40
first_iterations = []
last_iterations = []

the.branch = False
random.seed(10)

# Run trials
for trial in range(100):
    print(f'\nTrial {trial}')
    random.shuffle(d.rows)
    result, iteration_results = activeLearning(d, lambda B, R: exploit(B, R))

    if iteration_results:  # Store the first and last iteration results
        first_iterations.append(iteration_results[0])
        last_iterations.append(iteration_results[-1])

# Calculate and print the average values
average_first = sum(first_iterations) / len(first_iterations)
average_last = sum(last_iterations) / len(last_iterations)

print(f'\nAverage value of the first iteration across 100 trials: {average_first}')
print(f'Average value of the last iteration across 100 trials: {average_last}')

data/optimize/misc/auto93.csv
398

Trial 0
0. 91
1. 94
2. 94
3. 91
4. 91
5. 91
6. 91
7. 91
8. 91
9. 94
10. 94
11. 95
12. 94
13. 94
14. 94
15. 94
16. 94
17. 91
18. 91
19. 91
20. 91
21. 91
22. 91
23. 91
24. 91
25. 91
26. 91
27. 94
28. 94
29. 94
30. 94
31. 94
32. 91
33. 91
34. 91
35. 95

Trial 1
0. 23
1. 13
2. 13
3. 10
4. 10
5. 8
6. 23
7. 23
8. 19
9. 10
10. 23
11. 23
12. 24
13. 24
14. 24
15. 24
16. 29
17. 19
18. 19
19. 19
20. 19
21. 19
22. 19
23. 19
24. 19
25. 19
26. 19
27. 20
28. 20
29. 20
30. 20
31. 15
32. 15
33. 15
34. 15
35. 15

Trial 2
0. 95
1. 95
2. 95
3. 95
4. 95
5. 95
6. 95
7. 93
8. 71
9. 92
10. 92
11. 92
12. 92
13. 92
14. 92
15. 92
16. 92
17. 92
18. 92
19. 92
20. 92
21. 92
22. 92
23. 92
24. 92
25. 92
26. 92
27. 93
28. 93
29. 93
30. 93
31. 93
32. 93
33. 93
34. 93
35. 93

Trial 3
0. 56
1. 93
2. 93
3. 90
4. 90
5. 80
6. 80
7. 80
8. 80
9. 81
10. 80
11. 82
12. 82
13. 82
14. 82
15. 82
16. 82
17. 81
18. 81
19. 81
20. 81
21. 81
22. 82
23. 82
24. 82
25. 82
26. 82
27. 81
28. 81
29. 81
30. 8

# Exploit with RRP

In [13]:
# Load dataset and initialize
dataset = 'data/optimize/misc/auto93.csv'
d = DATA().adds(csv(dataset))

print(dataset)
print(len(d.rows))

def exploit(B, R):
    return B

def explore(B, R):
    return (B + R) / (abs(B - R) + 10 ** -30)

the.Last = 40
first_iterations = []
last_iterations = []

the.branch = True
random.seed(10)

# Run trials
for trial in range(100):
    print(f'\nTrial {trial}')
    random.shuffle(d.rows)
    result, iteration_results = activeLearning(d, lambda B, R: exploit(B, R))

    if iteration_results:  # Store the first and last iteration results
        first_iterations.append(iteration_results[0])
        last_iterations.append(iteration_results[-1])

# Calculate and print the average values
average_first = sum(first_iterations) / len(first_iterations)
average_last = sum(last_iterations) / len(last_iterations)

print(f'\nAverage value of the first iteration across 100 trials: {average_first}')
print(f'Average value of the last iteration across 100 trials: {average_last}')

data/optimize/misc/auto93.csv
398

Trial 0
4. 57
5. 95
6. 95
7. 94
8. 94
9. 94
10. 93
11. 93
12. 94
13. 94
14. 94
15. 94
16. 94
17. 94
18. 94
19. 94
20. 94
21. 94
22. 94
23. 94
24. 94
25. 94
26. 94
27. 94
28. 94
29. 94
30. 94
31. 94
32. 94
33. 94
34. 95
35. 95

Trial 1
4. 42
5. 92
6. 94
7. 94
8. 94
9. 94
10. 94
11. 94
12. 94
13. 94
14. 94
15. 94
16. 94
17. 94
18. 94
19. 94
20. 94
21. 94
22. 94
23. 94
24. 94
25. 94
26. 94
27. 94
28. 94
29. 94
30. 94
31. 94
32. 94
33. 94
34. 94
35. 94

Trial 2
4. 46
5. 82
6. 82
7. 82
8. 90
9. 79
10. 79
11. 90
12. 90
13. 90
14. 90
15. 86
16. 86
17. 86
18. 86
19. 86
20. 86
21. 86
22. 85
23. 84
24. 84
25. 84
26. 84
27. 86
28. 86
29. 86
30. 86
31. 86
32. 86
33. 86
34. 86
35. 86

Trial 3
4. 49
5. 91
6. 91
7. 91
8. 91
9. 91
10. 91
11. 91
12. 91
13. 91
14. 91
15. 91
16. 91
17. 91
18. 91
19. 91
20. 91
21. 91
22. 91
23. 91
24. 91
25. 91
26. 92
27. 92
28. 92
29. 93
30. 93
31. 95
32. 95
33. 95
34. 95
35. 94

Trial 4
4. 42
5. 94
6. 94
7. 94
8. 94
9. 92
10. 92
11. 92

# Explore with Random Initialization

In [15]:
# Load dataset and initialize
dataset = 'data/optimize/misc/auto93.csv'
d = DATA().adds(csv(dataset))

print(dataset)
print(len(d.rows))

def exploit(B, R):
    return B

def explore(B, R):
    return (B + R) / (abs(B - R) + 10 ** -30)

the.Last = 40
first_iterations = []
last_iterations = []

the.branch = False
random.seed(10)

# Run trials
for trial in range(100):
    print(f'\nTrial {trial}')
    random.shuffle(d.rows)
    result, iteration_results = activeLearning(d, lambda B, R: explore(B, R))

    if iteration_results:  # Store the first and last iteration results
        first_iterations.append(iteration_results[0])
        last_iterations.append(iteration_results[-1])

# Calculate and print the average values
average_first = sum(first_iterations) / len(first_iterations)
average_last = sum(last_iterations) / len(last_iterations)

print(f'\nAverage value of the first iteration across 100 trials: {average_first}')
print(f'Average value of the last iteration across 100 trials: {average_last}')

data/optimize/misc/auto93.csv
398

Trial 0
0. 91
1. 91
2. 91
3. 68
4. 84
5. 84
6. 84
7. 84
8. 84
9. 72
10. 83
11. 83
12. 83
13. 83
14. 83
15. 83
16. 83
17. 72
18. 83
19. 85
20. 85
21. 85
22. 85
23. 85
24. 85
25. 85
26. 85
27. 84
28. 84
29. 84
30. 84
31. 85
32. 85
33. 85
34. 85
35. 85

Trial 1
0. 23
1. 23
2. 24
3. 11
4. 35
5. 34
6. 34
7. 34
8. 38
9. 40
10. 40
11. 40
12. 40
13. 40
14. 40
15. 41
16. 41
17. 23
18. 47
19. 52
20. 52
21. 52
22. 74
23. 74
24. 74
25. 74
26. 74
27. 62
28. 61
29. 61
30. 61
31. 59
32. 59
33. 57
34. 57
35. 57

Trial 2
0. 95
1. 95
2. 95
3. 95
4. 95
5. 95
6. 95
7. 95
8. 95
9. 59
10. 92
11. 92
12. 92
13. 92
14. 92
15. 92
16. 92
17. 63
18. 95
19. 95
20. 95
21. 95
22. 95
23. 95
24. 95
25. 95
26. 95
27. 65
28. 94
29. 94
30. 94
31. 94
32. 94
33. 94
34. 94
35. 94

Trial 3
0. 56
1. 55
2. 52
3. 79
4. 79
5. 79
6. 78
7. 77
8. 74
9. 79
10. 79
11. 79
12. 79
13. 80
14. 80
15. 80
16. 80
17. 82
18. 80
19. 83
20. 83
21. 83
22. 83
23. 83
24. 83
25. 83
26. 83
27. 81
28. 81
29. 81
30. 

In [16]:
# Load dataset and initialize
dataset = 'data/optimize/misc/auto93.csv'
d = DATA().adds(csv(dataset))

print(dataset)
print(len(d.rows))

def exploit(B, R):
    return B

def explore(B, R):
    return (B + R) / (abs(B - R) + 10 ** -30)

the.Last = 40
first_iterations = []
last_iterations = []

the.branch = True
random.seed(10)

# Run trials
for trial in range(100):
    print(f'\nTrial {trial}')
    random.shuffle(d.rows)
    result, iteration_results = activeLearning(d, lambda B, R: explore(B, R))

    if iteration_results:  # Store the first and last iteration results
        first_iterations.append(iteration_results[0])
        last_iterations.append(iteration_results[-1])

# Calculate and print the average values
average_first = sum(first_iterations) / len(first_iterations)
average_last = sum(last_iterations) / len(last_iterations)

print(f'\nAverage value of the first iteration across 100 trials: {average_first}')
print(f'Average value of the last iteration across 100 trials: {average_last}')

data/optimize/misc/auto93.csv
398

Trial 0
4. 57
5. 95
6. 95
7. 95
8. 95
9. 94
10. 94
11. 94
12. 94
13. 94
14. 94
15. 94
16. 94
17. 93
18. 93
19. 93
20. 93
21. 93
22. 93
23. 93
24. 93
25. 93
26. 93
27. 95
28. 95
29. 95
30. 95
31. 95
32. 95
33. 95
34. 95
35. 95

Trial 1
4. 42
5. 92
6. 92
7. 92
8. 92
9. 90
10. 90
11. 90
12. 90
13. 90
14. 90
15. 90
16. 90
17. 95
18. 95
19. 95
20. 95
21. 95
22. 95
23. 95
24. 95
25. 95
26. 95
27. 91
28. 91
29. 91
30. 91
31. 91
32. 91
33. 91
34. 91
35. 91

Trial 2
4. 46
5. 95
6. 95
7. 95
8. 95
9. 92
10. 92
11. 95
12. 95
13. 95
14. 95
15. 95
16. 95
17. 95
18. 95
19. 95
20. 95
21. 95
22. 95
23. 95
24. 95
25. 95
26. 95
27. 92
28. 92
29. 92
30. 92
31. 92
32. 92
33. 92
34. 92
35. 92

Trial 3
4. 49
5. 91
6. 91
7. 91
8. 91
9. 91
10. 91
11. 91
12. 91
13. 91
14. 91
15. 91
16. 91
17. 93
18. 93
19. 93
20. 93
21. 93
22. 93
23. 93
24. 93
25. 93
26. 93
27. 93
28. 93
29. 93
30. 93
31. 93
32. 93
33. 93
34. 93
35. 93

Trial 4
4. 42
5. 94
6. 95
7. 95
8. 95
9. 94
10. 94
11. 94

# Results Summary

Exploit with random: 73.93 -> 79.16
Exploit with rrp:    51.54 -> 92.18
Explore with random: 73.93 -> 84.61
Explore with rrp:    51.54 -> 90.07

UCB with random:     80.26 -> 79.50
UCB with rrp:        80.25 -> 78.34